In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
from tqdm import tqdm
import json
pd.set_option('display.max_columns', None)
%load_ext autoreload
%autoreload 2
import sys
sys.path.append('../boat/')
sys.path.append('../')
import pickle

In [2]:
with open('../boat/config.json', 'r') as f:
    config = json.load(f)
USE_COLS = config['pred']['use_cols']
FRM = config['pred']['frm']
PL_CODE = config['場所']
print(FRM)
print(USE_COLS)

15
['RaceID', '場所', 'R', '艇番', '今節成績11', '今節成績12', '今節成績21', '今節成績22', '今節成績31', '今節成績32', '今節成績41', '今節成績42', '今節成績51', '今節成績52', '今節成績61', '今節成績62', '着', '選手登番', '体重', '性別', '級別', '全国勝率', '全国2連率', '当地勝率', '当地2連率', 'モーター2連率', 'ボート2連率', '今期能力指数', '前期能力指数', '1着回数', '2着回数', '逃げ率', 'まくり率', '平均スタートタイミング', '出走回数', '複勝率', '勝率', '1コース1着回数', '2コース1着回数', '3コース1着回数', '4コース1着回数', '5コース1着回数', '6コース1着回数', '1コースまくられ率']


In [3]:
from config import *

In [4]:
set_values(globals())

True

In [14]:
pred = pd.read_csv(f'../pred/pred_{FRM}.csv')

In [19]:
races = pred['RaceID'].unique()
df_res = pd.DataFrame()
win = 0
print(len(races))
n = 0
bet = 0
for m, raceid in enumerate(races):
    temp = pd.DataFrame()
    race = pred.query('RaceID == @raceid')
    if len(race) != 6:
        continue
    p = (race['pred'].argsort()[::-1] + 1).values
    place = race['場所'].values[0]

    
    #nige = race['逃げ率'].values[0] > 0.4
    nige_s = race.query(f'艇番 == {p[0]}')['逃げ率'].values[0]
    nige = race.query(f'艇番 == {p[0]}')['逃げ率'].values[0] > 0.6
    female = sum(race['性別'].values) == 12
    ana = p[0] != 1
    #pl = place in [21,24,19]
    #pl = place in [3,2,4,8,5]
    pl = True
    if  pl :
        buy = True
        n += 1
    else:
        buy = False
        
    if buy:
        path = f'../data/csv/Race_info/K{str(raceid)[2:8]}.json'
        try:
            with open(path) as f:
                data = json.load(f)
            res = data[f'{raceid}']['3連単']['result']
            prize = int(data[f'{raceid}']['3連単']['prize'])

            pred_rank = ''
            for i in p:
                pred_rank += str(i)
            kaime = []
            kaime.append(pred_rank[:3])
            '''
            kaime.append(pred_rank[0] + pred_rank[2] + pred_rank[1])
            kaime.append(pred_rank[1] + pred_rank[0] + pred_rank[2])
            '''
            kaime.append(pred_rank[:2] + pred_rank[3])
            kaime.append(pred_rank[:2] + pred_rank[4])
            kaime.append(pred_rank[:2] + pred_rank[5])
            
            temp['RaceID'] = raceid
            temp['pred_rank'] = pred_rank
            temp['result'] = res
            temp['prize'] = prize
            temp = pd.DataFrame({'RaceID' : [raceid], 'pred_rank' : [pred_rank], 'result' : [res], 'prize' : [prize]})
            df_res = pd.concat([df_res, temp])
            
            bet += 100 * len(kaime)
            if res.replace('-','')[:3] in kaime:
                
                win += prize 
                win_rate = win / bet * 100
                print(f'Np{n} win. Place {place}. Prize is {prize}. Win rate is {win_rate:.2f}%, nigero is {nige_s:.2f}')
            else:
                win_rate = win / bet * 100
                #print(f'Np{n} lose. Place {place}.')
                print(f"No{n} lose. Place {place}. Nige {nige_s:.2f}, Win rate is {win_rate:.2f}%")
                pass
                #print(f'{kaime} != {res.replace("-","")[:3]}')
                #print(f'RaceID:{raceid} is incorrect. Prize is {prize}. Win is {win - 100}')
        except KeyboardInterrupt:
            break
        except Exception as e:
            #import traceback; traceback.print_exc();
            
            #print(e)
            pass
df_res = df_res.reset_index(drop=True)

924
No1 lose. Place 23. Nige 0.12, Win rate is 0.00%
No2 lose. Place 23. Nige 0.14, Win rate is 0.00%
Np3 win. Place 23. Prize is 960. Win rate is 80.00%, nigero is 0.11
No4 lose. Place 23. Nige 0.76, Win rate is 60.00%
No5 lose. Place 23. Nige 0.26, Win rate is 48.00%
Np6 win. Place 23. Prize is 550. Win rate is 62.92%, nigero is 0.38
Np7 win. Place 23. Prize is 1130. Win rate is 94.29%, nigero is 0.77
Np8 win. Place 23. Prize is 640. Win rate is 102.50%, nigero is 0.33
No9 lose. Place 23. Nige 0.35, Win rate is 91.11%
No10 lose. Place 23. Nige 0.69, Win rate is 82.00%
No11 lose. Place 23. Nige 0.68, Win rate is 74.55%
No12 lose. Place 23. Nige 0.74, Win rate is 68.33%
No13 lose. Place 21. Nige 0.77, Win rate is 63.08%
No14 lose. Place 21. Nige 0.93, Win rate is 58.57%
No15 lose. Place 21. Nige 0.83, Win rate is 54.67%
No16 lose. Place 21. Nige 0.80, Win rate is 51.25%
No17 lose. Place 21. Nige 0.71, Win rate is 48.24%
Np18 win. Place 21. Prize is 1970. Win rate is 72.92%, nigero is 0

In [ ]:
pred_all = df_res

In [29]:
from datetime import datetime as dt

today = dt.now().strftime('%Y%m%d')


In [30]:
today

'20240823'

In [ ]:
pred.query('RaceID == 202308130109')

,RaceID,場所,R,艇番,今節成績11,今節成績12,今節成績21,今節成績22,今節成績31,今節成績32,今節成績41,今節成績42,今節成績51,今節成績52,今節成績61,今節成績62,着,選手登番,体重,性別,級別,全国勝率,全国2連率,当地勝率,当地2連率,モーター2連率,ボート2連率,今期能力指数,前期能力指数,1着回数,2着回数,逃げ率,まくり率,平均スタートタイミング,出走回数,複勝率,勝率,1コース1着回数,2コース1着回数,3コース1着回数,4コース1着回数,5コース1着回数,6コース1着回数,1コースまくられ率,pred
0,202308130109,1,9,6,0,0,0,0,0,0,0,0,0,0,0,0,4,4539,52,1,3,5.93,43.14,5.85,41.07,37.78,33.78,5200,5600,28,14,0.875,0.152778,15,103,407,573,14,3,4,7,0,0,0.0625,-0.419822


In [ ]:
pred_all

,RaceID,pred_rank,result,prize
0,202308130109,1,1-2-5,2970
1,202308130110,124536,1-2-3,1580
2,202308130111,124536,2-1-4,1590
3,202308130112,132456,1-5-3,2800
4,202308142401,124536,5-1-3,10810
...,...,...,...,...
48532,202407060108,152436,5-1-2,980
48533,202407060109,215346,2-1-4,7370
48534,202407060110,123465,1-6-3,1800
48535,202407060111,124356,2-3-4,560
